In [3]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [5]:



# url_template = "https://thuvienphapluat.vn/page/tim-van-ban.aspx?keyword=m%C3%A3+s%E1%BB%91&area=0&match=True&type=21&status=0&signer=0&sort=1&lan=1&scan=0&org=0&fields=4&page={}"


# def get_links_from_page(page_number):
#     url = url_template.format(page_number)
#     response = requests.get(url)
    

#     if response.status_code != 200:
#         print(f"Lỗi khi tải trang {page_number}")
#         return []


#     soup = BeautifulSoup(response.text, 'html.parser')


#     divs = soup.find_all('div', class_=re.compile(r'content-[01]'))


#     links = [div.find('a')['href'] for div in divs if div.find('a')]
    
#     return links


# all_links = []
# for page in range(1, 401):
#     print(f"Đang tải trang {page}")
#     links = get_links_from_page(page)
#     all_links.extend(links)

# print(f"Tổng số link thu thập được: {len(all_links)}")


In [6]:
# with open('links.txt', 'w', encoding='utf-8') as f:
#     for link in all_links:
#         f.write(link + '\n')

# print("Ghi thành công các link vào file 'links.txt'")

In [4]:
# thay list link cần chạy

links_path = os.path.join(current_directory, 'tesst_link.txt')

with open(links_path, 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [5]:
len(all_links)

65

In [6]:
result_path = r"C:\Users\nguye\Desktop\result.xlsx"
workbook = load_workbook(result_path)
maso_sheet = workbook['Sheet1']
phanloai_sheet = workbook['Sheet2']

In [ ]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text(strip=True).replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
def write_data_to_excel(sheets_name,info_list):   
# Ghi dữ liệu vào dòng mới
    row = sheets_name.max_row + 1
    for col, value in enumerate(info_list, start=1):
        sheets_name.cell(row=row, column=col, value=value)

    # Lưu workbook
    workbook.save(result_path)


In [10]:

def get_ma_so_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')
    part1_part_1  = next((i for i, item in enumerate(part1) if "1. Hàng hóa " in item.get_text(strip=True)), -1)
    part1_part_1_text  = part1[part1_part_1].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_2 = next((i for i, item in enumerate(part1) if "Tên thương mại" in item.get_text(strip=True)), -1)
    part1_part_2_text  = part1[part1_part_2].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_3 = next((i for i, item in enumerate(part1) if "Tên gọi theo cấu tạo" in item.get_text(strip=True)), -1)
    part1_part_3_text  = part1[part1_part_3].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_4 = next((i for i, item in enumerate(part1) if "mã hiệu" in item.get_text(strip=True)), -1)
    part1_part_4_text  = part1[part1_part_4].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    part1_part_5 = next((i for i, item in enumerate(part1) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
    part1_part_5_text  = part1[part1_part_5].get_text(strip=True).replace('\r', '').replace('\n', '').strip()
    info_list.append(part1_part_2_text)
    info_list.append(part1_part_3_text)
    info_list.append(part1_part_4_text)
    info_list.append(part1_part_5_text)

    # allpart
    allpart = tables[1].find_all('p')

    # lấy part 3
    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in allpart:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part3 = matches[-1]
    # lấy part 2
    try :
        part2_start = next((i for i, item in enumerate(allpart) if "2. Tóm tắt" in item.get_text(strip=True)), -1)-1
        # print(part2_start)
    except:
        part2_start = next((i for i, item in enumerate(allpart) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
        # print(part2_start)
    part2_end = next((i for i, item in enumerate(allpart) if "3. Kết quả" in item.get_text(strip=True)), -1)
    part2_list = []
    for i in allpart[part2_start+1:part2_end]:
        part2_list.append(i.get_text(strip=True))
    part2_list
    part2_text = '\n'.join(part2_list)

    info_list.append(part2_text)
    info_list.append(part3)
    print(info_list)
    return info_list

In [12]:
def get_phanloai_link_info(link):
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # Khởi tạo các biến trước để tránh lỗi khi không tìm thấy
    số = "Không tìm thấy 'Số'"
    ngày = "Không tìm thấy 'Ngày'"
    part_1_text = "Không tìm thấy part 1"
    part_2_text = "Không tìm thấy part 2"
    part_3_text = "Không tìm thấy part 3"
    part_4_text = "Không tìm thấy part 4"
    part_5_text = "Không tìm thấy thông tin phần 5"

    # Lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'Số'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'Số': {e}")

    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text or 'Ngày' in text:
                ngày = normalize_date(text)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'ngày': {e}")

    # Lấy part 1 (bảng thứ 2 đầu của trang)
    part1 = tables[1].find_all('p')
    # Tạo re để tìm chuỗi theo các phần       
    part1_1_pattern = re.compile(r"1\.\s*[Tt]ên\s*[Hh]àng")
    part1_2_pattern = re.compile(r"2\.\s*Đơn\s*vị")
    part1_3_pattern = re.compile(r"3\.\s*Số")
    part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
    part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
    part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")

    # Tìm index các phần
    part1_1_index = next((i for i, p_tag in enumerate(part1) if part1_1_pattern.search(p_tag.get_text(strip=True))), None)
    part1_2_index = next((i for i, p_tag in enumerate(part1) if part1_2_pattern.search(p_tag.get_text(strip=True))), None)
    part1_3_index = next((i for i, p_tag in enumerate(part1) if part1_3_pattern.search(p_tag.get_text(strip=True)) or part1_3_pattern_2.search(p_tag.get_text(strip=True))), None)
    part1_4_index = next((i for i, p_tag in enumerate(part1) if part1_4_pattern.search(p_tag.get_text(strip=True))), None)
    part1_5_index = next((i for i, p_tag in enumerate(part1) if part1_5_pattern.search(p_tag.get_text(strip=True))), None)

    # Kiểm tra nếu các chỉ mục có tồn tại, nếu không gán giá trị thông báo không tìm thấy mặc định
    if part1_1_index is not None and part1_2_index is not None:
        part_1_text = part_text(part1, part1_1_index, part1_2_index)
    if part1_2_index is not None and part1_3_index is not None:
        part_2_text = part_text(part1, part1_2_index, part1_3_index)
    if part1_3_index is not None and part1_4_index is not None:
        part_3_text = part_text(part1, part1_3_index, part1_4_index)
    if part1_4_index is not None and part1_5_index is not None:
        part_4_text = part_text(part1, part1_4_index, part1_5_index)

    # Lấy thông tin phần 5 nếu tìm thấy index
    if part1_5_index is not None:
        pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
        for p_tag in part1[part1_5_index:]:
            text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
            matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
            if matches:
                part_5_text = matches[-1]  # Lấy kết quả cuối cùng khớp với pattern

    # Tạo danh sách chứa thông tin
    info_list = [link.strip(), số, ngày, part_1_text, part_2_text, part_3_text, part_4_text, part_5_text]
    print(link)
    print(info_list)
    return info_list


In [ ]:
# all_maso_info_list = []
# for link in all_links:
#     if 'truoc-ma-so' in link:
#         try:
#             info_list = get_ma_so_link_info(link)
#             all_maso_info_list.append(info_list)
#             write_data_to_excel(maso_sheet,info_list)
#         except:
#             print(link + "lỗi")
#             info_list = [link,"lỗi"]
#             all_maso_info_list.append([link,"lỗi"])
#             write_data_to_excel(phanloai_sheet,info_list)

In [14]:
all_phanloai_info_list=[]
for link in all_links:
    if 'phan-loai' in link:
        try:
            info_list = get_phanloai_link_info(link)
            all_phanloai_info_list.append(info_list)
            write_data_to_excel(phanloai_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_phanloai_info_list.append(info_list)
            write_data_to_excel(phanloai_sheet,info_list)

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-87-TB-TCHQ-phan-loai-hang-hoa-xuat-nhap-khau-vai-thun-2016-317336.aspx

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-87-TB-TCHQ-phan-loai-hang-hoa-xuat-nhap-khau-vai-thun-2016-317336.aspx', '87/TB-TCHQ', '06-01-2016', '1. Tên hàng theo khai báo:Vải thun (theokết quảchứng  thư giám định tìnhtrạng số 15G10ND00496-301 ngày 18/9/2015) hàng chưa qua sử dụng. (Mục 1 Phụ lục Tờ khai)', '2. Đơnvị nhập khẩu:Công ty  cổ phần Thiết bị và Vật tư Miền Nam - Địa chỉ: 259 Lê Văn Lương, phường Tân  Quy, Quận 7, TP. Hồ Chí Minh. MST: 0312057516.', '3. Tờ khai số:10059048581/A11 ngày 14/10/2015 đăng ký tại Chi cục HQ CK Cảng Cái  Mép-Cục Hải quan tỉnh BàRịa -  Vũng Tàu.', '4. Tóm tắt mô tả và đặc tính hàng hóa:Vải dệt kim sợi ngang từ bông, đã in các sọc màu, trọng lượng vải =  204 g/m2.', '6006.24.00']
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-95-TB-TCHQ-phan-loai-hang-hoa-xuat-nhap-khau-Quang-Zircon-2016-317

In [18]:
# with open('output.txt', 'w') as f:
#     # Duyệt qua từng list con và ghi từng dòng vào file
#     for row in all_phanloai_info_list:
#         # Chuyển list con thành chuỗi với các phần tử ngăn cách bởi dấu phẩy
#         row_str = ', '.join(map(str, row))
#         # Ghi vào file và xuống dòng
#         f.write(row_str + '\n')

In [19]:
# wb2 = Workbook()
# ws2 = wb2.active  # Lấy sheet mặc định

# # Ghi từng dòng của danh sách vào các ô Excel
# for row in all_phanloai_info_list:
#     ws2.append(row)  # Append từng list con vào sheet

# # Lưu workbook vào file Excel
# wb2.save("output.xlsx")

# print("Danh sách hai chiều đã được ghi vào file 'output.xlsx'.")